***
### Hands-on : GPT
***
* We will use the GPT-2 model for NLG/Language Modelling
* Try seeding various sentence fragments and see what it comes up with

* Was OpenAI justified in staged release ?
* Has our new overlords reached a stage where they can fool us ?

In [1]:
import torch
print("torch ver :",torch.__version__)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#
# pip install https://github.com/pytorch/text/archive/master.zip
import torchtext
print(F'TorchText Ver : {torchtext.__version__}')
from torchtext import data
#
import matplotlib.pyplot as plt
%matplotlib inline
from torchsummary import summary
# pip install torchsummary
#
import numpy as np
import datetime # use datetime.datetime.now() prints in a nice hh:mm:ss.nn forma
import time

torch ver : 1.4.0
TorchText Ver : 0.5.1


In [2]:
is_cuda = False
if torch.cuda.is_available():
    is_cuda = True
print('Cuda : {:.1s}'.format(str(is_cuda)))

Cuda : F


In [3]:
# Huggingface Transformers
# pip install transformers
# Might need rust compiler
# curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
# need Cargo's bin directory ($HOME/.cargo/bin) in your PATH environment variable
import transformers
# The GPT2 Model transformer with a language modeling head on top 
#   (linear layer with weights tied to the input embeddings).
from transformers import GPT2Tokenizer, GPT2LMHeadModel

I0221 20:58:15.685184 4640626112 file_utils.py:41] PyTorch version 1.4.0 available.
/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/miniconda3/lib/python3.7/site-

In [4]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

I0221 20:58:22.202440 4640626112 tokenization_utils.py:484] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /Users/ksankar/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
I0221 20:58:22.203521 4640626112 tokenization_utils.py:484] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /Users/ksankar/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
I0221 20:58:22.759586 4640626112 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /Users/ksankar/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.699bbd1c449e9861456f359d6daa51bd523ac085b4b531ab0aad5a55d091e942
I0221 

`Model config GPT2Config {
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "embd_pdrop": 0.1,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_beams": 1,
  "num_labels": 2,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "pruned_heads": {},
  "repetition_penalty": 1.0,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "temperature": 1.0,
  "top_k": 50,
  "top_p": 1.0,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 50257
}
`

In [6]:
print(gpt2_tokenizer.vocab_size)
input_text = gpt2_tokenizer.encode("We like Unicorns because they")
print(gpt2_tokenizer.convert_ids_to_tokens(input_text))
input_text = gpt2_tokenizer.encode("Here is the sentence I want embeddings for.")
print(gpt2_tokenizer.convert_ids_to_tokens(input_text))

50257
['We', 'Ġlike', 'ĠUnic', 'orns', 'Ġbecause', 'Ġthey']
['Here', 'Ġis', 'Ġthe', 'Ġsentence', 'ĠI', 'Ġwant', 'Ġembed', 'd', 'ings', 'Ġfor', '.']


In [7]:
def run_gpt2_sentence_prediction(text,n=50):
    input_text = gpt2_tokenizer.encode(text)
    input,past = torch.tensor([input_text]),None
    for _ in range(n):#25):
        logits,past = gpt2_model(input,past=past)
        input = torch.multinomial(F.softmax(logits[:,-1]),1)
        input_text.append(input.item())
    return gpt2_tokenizer.decode(input_text)

In [8]:
in_text = "We like Unicorns because they"
out = run_gpt2_sentence_prediction(in_text)
print(out)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


We like Unicorns because they have fast baneling fury, hard pressure and ward.


Statistics


Level 1: 2368 (78 Rank)


Experiment #13: Join Alliance

by Gorgeius (nice hack)


Level 1: 250 (


### Output from a run on 2/21/20 7:00 PM

`We like Unicorns because they are committed to building the most progressive platform so no major issues are going to come up again.

The race against Unicorns has become a proving ground for champion performance in eSports, and the local leagues have found greater strength.

Leaguepedia`

In [10]:
in_text = "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously \
unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the \
unicorns spoke perfect English."
out = run_gpt2_sentence_prediction(in_text,100)
print(out)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

After swimming alone in a lake for just under a week north of Bolivia's capital Puebla and chowing down face-to-face with some large wilds (save for some small more common frachy-huana like chillalone) and other lifeform (such as exotic jellyfish with crocodilian features) among the nearly 2.6 million other natural task animals on Earth from animal to animal, Chi- Chi also found that a number of minnows and some domesticated


In [11]:
in_text = "You're in a desert, walking along in the sand, when all of a sudden you look down and see a tortoise,\
Leon. It's crawling toward you. You reach down, you flip the tortoise over on its back.\
The tortoise lays on its back, its belly baking in the hot sun, beating its legs trying to\
turn itself over,but it can’t, not without your help. But you’re not helping. Why is that?"
out = run_gpt2_sentence_prediction(in_text,200)
print(out)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


You're in a desert, walking along in the sand, when all of a sudden you look down and see a tortoise,Leon. It's crawling toward you. You reach down, you flip the tortoise over on its back.The tortoise lays on its back, its belly baking in the hot sun, beating its legs trying toturn itself over,but it can’t, not without your help. But you’re not helping. Why is that? You see to it that it is not starved‼This is why‬this is why. Because when you think about․they see your mouth,‼they think‼she could reach․this way․and‼how‼but‼she‼you‬can› pull․you out‼very quickly‼because she falters in the hot sun and›it''s**you‬ere leaking oil.Wolf - If you lean against her while resting and gaze at her palms with her not intending‼to–she‼is‹resting on her knees,‼but if she doesn't take milk orcheese?›Does she? You don´t know?Wife‼proud‼looking at me with commanding eyes.Daughter standing facing me:‼how about‼working‼daddy on her knees,Anna catches her breath:Putting


### Homework:
* This is an open topic to explore
* Try your own text and see how the GPT responds
* Try longer sentences - input and output - I have capped the output at 200 words
* Do you get better longer response with an input with longer sentences ?
* How many times do you have to do before getting an output that is interesting ?
* Does the model give coherent output at the first try ?

In [ ]:
# in_text = ""
# output_length = 200
# out = run_gpt2_sentence_prediction(in_text,output_length)
# print(out)

### You can even write a whole book with the help from transformer [here](https://transformer.huggingface.co/)
![](Write-with-Transformer.png)


***
## _That's All Folks !_
***

## WIP area
### To experiment with different things 

* summary() doesn't work well
* you can print(model) to get an idea about the layers
* The model_describe() below works reasonably, but the total parameters doesn't look correct
* Let me know if you were able to get this to work correctly

In [9]:
# summary(gpt2_model,[5])

In [10]:
def model_describe(model):
    print("model_summary")
    print()
    print(F'Layer_name {"":30s} Size {"":25s} Number of Parameters')
    print("="*100)
    total_params = 0
    for param_tensor in model.state_dict():
        print(F'Layer {param_tensor:35s} {str(model.state_dict()[param_tensor].size()):30s} '
              F'elements : {torch.numel(model.state_dict()[param_tensor]):,}')
        total_params += torch.numel(model.state_dict()[param_tensor])
    # total_params = sum(p.numel() for p in model.parameters()) # sum number of elements
    print("="*100)
    print(f"Total Params:{total_params:,}")

In [11]:
model_describe(gpt2_model)

model_summary

Layer_name                                Size                           Number of Parameters
Layer transformer.wte.weight              torch.Size([50257, 768])       elements : 38,597,376
Layer transformer.wpe.weight              torch.Size([1024, 768])        elements : 786,432
Layer transformer.h.0.ln_1.weight         torch.Size([768])              elements : 768
Layer transformer.h.0.ln_1.bias           torch.Size([768])              elements : 768
Layer transformer.h.0.attn.bias           torch.Size([1, 1, 1024, 1024]) elements : 1,048,576
Layer transformer.h.0.attn.c_attn.weight  torch.Size([768, 2304])        elements : 1,769,472
Layer transformer.h.0.attn.c_attn.bias    torch.Size([2304])             elements : 2,304
Layer transformer.h.0.attn.c_proj.weight  torch.Size([768, 768])         elements : 589,824
Layer transformer.h.0.attn.c_proj.bias    torch.Size([768])              elements : 768
Layer transformer.h.0.ln_2.weight         torch.Size([768])           

In [12]:
print(gpt2_model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [13]:
def model_summary(model): # This calculates wrong, may be it is correct - need to check
  print("model_summary")
  print()
  print("Layer_name"+"\t"*7+"Number of Parameters")
  print("="*100)
  model_parameters = [layer for layer in model.parameters() if layer.requires_grad]
  layer_name = [child for child in model.children()]
  j = 0
  total_params = 0
  print("\t"*10)
  for i in layer_name:
    print()
    param = 0
    try:
      bias = (i.bias is not None)
    except:
      bias = False  
    if not bias:
      param =model_parameters[j].numel()+model_parameters[j+1].numel()
      j = j+2
    else:
      param =model_parameters[j].numel()
      j = j+1
    print(str(i)+"\t"*3+str(param))
    total_params+=param
  print("="*100)
  print(f"Total Params:{total_params}")       

model_summary(gpt2_model)

model_summary

Layer_name							Number of Parameters
										

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, 

In [ ]:
# GPT2 Playground
# https://github.com/ilopezfr/gpt-2
# Colab Playground 
# Check out https://colab.research.google.com/github/ilopezfr/gpt-2/blob/master/gpt-2-playground_.ipynb

### Handy code for testing elapsed time. It prints hh:mm:ss.nnn correctly. time.time() prints the total seconds

In [62]:
import datetime
import time
start_time = datetime.datetime.now()
time.sleep(145)
print(F'Elapsed - {datetime.datetime.now() - start_time}')

Elapsed - 0:02:25.006721
